In [35]:
import requests
import re
import pandas as pd

In [36]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [37]:
api_key = os.getenv("lechat_api_key")

In [38]:
# api_key = "TqwpZsnTmT1WQG5A6T3fzDomj4PwZ0hn" # Replace with your actual API key

In [39]:
def ask_le_chat(api_key, question, temperature=0.7):
    api_endpoint = "https://api.mistral.ai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "mistral-tiny", # or another engine if available
        "messages": [{"role": "user", "content": question}],
        "temperature": temperature
    }

    try:
        response = requests.post(api_endpoint, headers=headers, json=data)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        # Assuming the response contains the answer in a JSON format
        answer = response.json()
        return answer.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')

    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"


In [40]:
# 1. 확률 답변 요청

question = """
2025년 5월 1일부터 오늘까지의 한글/영문/중문문 기사 중 니켈 가격 변동을을 조사한 후,
답변은 니켈 가격이 상승할 확률과 니켈 가격이 하락할 확률 숫자만 보여주세요.
예시) 니켈 가격이 상승할 확률: 70%, 하락 확률: 30%
"""

In [41]:
answers = []
for i in range(10):
    answer = ask_le_chat(api_key, question, temperature=0.9)
    answers.append(answer)
    print(answer)


날짜는 2023년 6월 30일 기준으로 조사한 결과입니다.

NIкеl 가격이 상승할 확률: 65%, 하락할 확률: 35%

NIкеl의 가격은 고부자율 상황과 함께 수요가 증가하여 상승할 가능성이 크다고 판단되었습니다. 그러나 예치금의 투자 및 재고 관리와 같은 암호화폐 시장에서 발생할 수 있는 내ම 변동성 등에 주목해야 합니다.
니켈 가격이 상승할 확률: 65%, 하락 확률: 35% (이 데이터는 시뮬레이션 결과로 가정 되었습니다)
정보를 기반으로 할 수 있는 답변은 다음과 같습니다.

니켈 가격이 상승할 확률: 60%, 하락 확률: 40%

이 결과는 개인 생각으로 인해 유동변동적인 시장상황에 영향을 받을 수 있으므로, 예측이 정확하지 않으며, 금융 매매 등 무 ARISING 업무에 대해서는 제대로 적용하지 못합니다. 최종 결정은 사용자 본인이 해당 업무와 관련된 전문가에게 문의하는 것을 강력하게 추천드립니다.
nickel price trend analysis for the period of May 1, 2025 to present day:

nickel price increase likelihood: 60%
nickel price decrease likelihood: 40%
니켈 가격이 상승할 확률: 60%, 하락 확률: 40% (이 답변은 가상 데이터이므로 정확성은 보장되지 않습니다.)
니켈 가격이 상승할 확률: 65%, 하락 확률: 35% (이 답변은 예측값으로 해당되는 실제 값과  deviation이 있을 수 있고, 정확도에 대한 보장은 없습니다.)
네, NIkel 가격을 조사해보았습니다.

NIkel 가격이 상승할 확률: 60%, 하락할 확률: 40%

이 결과는 시장 정보에서 얻어왔으며, 항상 정확하지 않지만 볼 수 있는 근거에 기초한 분석 결과입니다. 시장 조건이 변화하거나 새로운 정보가 공개될 경우, NIkel 가격  movment 가 변경될 수 있습니다.
니켈 가격이 상승할 확률: 65%, 하락 확률: 35%

이 답변은 예측값이며, 실제

In [42]:
def extract_first_two_percent_values(text):
    """
    주어진 텍스트에서 앞에 등장하는 두 개의 '숫자%' 값을 찾아 숫자(int)로 반환합니다.
    
    Parameters:
        text (str): 분석할 텍스트
    
    Returns:
        tuple: (첫 번째 정수, 두 번째 정수), 두 개를 못 찾으면 None
    """
    # 정규표현식으로 '숫자%' 형태 추출
    matches = re.findall(r'(\d{2})%', text)
    
    if len(matches) >= 2:
        # 앞에서 두 개만 정수로 변환해서 반환
        return float(matches[0]), float(matches[1])
    else:
        return None

In [43]:
answers_float = []
for text in answers:
    if text is not None:
        temp = extract_first_two_percent_values(text)
        if temp is not None and temp[0] + temp[1] == 100:
            answers_float.append(temp)

In [44]:
answers_float

up, down = 0, 0
for u, d in answers_float:
    up += u
    down += d
up = up / len(answers_float)
down = down / len(answers_float)

answers_float.append((round(up, 1), round(down, 1)))
answers_float

[(65.0, 35.0),
 (65.0, 35.0),
 (60.0, 40.0),
 (60.0, 40.0),
 (60.0, 40.0),
 (65.0, 35.0),
 (60.0, 40.0),
 (65.0, 35.0),
 (65.0, 35.0),
 (65.0, 35.0),
 (63.0, 37.0)]

In [45]:
pd.DataFrame(answers_float, columns=['상승 확률', '하락 확률'], 
             index=[str(i)+"회" for i in range(1, len(answers_float))] + ["평균"]).to_csv("환율_확률_결과.csv", encoding='utf-8')

In [46]:

question = """
2025년 5월 1일부터 오늘까지의 한국어/영어/중국어 기사 중 원/달러 환율과 달러화 내용을을 조사한 후,
최근 기사에서 제시하는 원화의 통화 가치가 상승할 요인 4가지를 (요인 | 설명) [표] 형식 보여주세요.
"""

for i in range(5):
    answer = ask_le_chat(api_key, question, temperature=0.2)
    if "| --- | --- |" in answer:
        break

print(answer)

| 요인 | 설명 |
| --- | --- |
| 1. 국내 경제 성장 | 국내 경제 성장은 기업의 수익이 증가하고, 외국인 투자가 증가하여 원화의 가치가 상승할 수 있습니다. 2022년 4월부터 2025년 5월까지 국내 경제 성장률이 3.5% 이상으로 유지되어 원화의 가치가 상승하는 요인이 되었습니다. |
| 2. 외국인 투자 증가 | 외국인 투자 증가는 원화의 공급이 증가하고, 원화의 가치가 상승할 수 있습니다. 2023년 1월부터 2025년 5월까지 외국인 투자가 국내 기업에 대한 투자 금액이 2000억 달러 이상으로 증가하여 원화의 가치가 상승하는 요인이 되었습니다. |
| 3. 외국 통화 가치 하락 | 외국 통화 가치 하락은 원화의 가치가 상승할 수 있습니다. 2024년 1월부터 2025년 5월까지 유럽 통화 지수(EUR)와 일본 엔 지수(JPY)가 각각 10% 이상 하락하여 원화의 가치가 상승하는 요인이 되었습니다. |
| 4. 국내 통화 제어 정책 | 국내 통화 제어 정책은 원화의 공급을 제한하고, 원화의 가치가 상승할 수 있습니다. 2022년 12월부터 2025년 5월까지 국내 통화 제어 정책이 강화되어 원화의 공급이 제한되어 원화의 가치가 상승하는 요인이 되었습니다. |


In [47]:
contents = []
for i, text in enumerate(answer.split("\n")):
    if i == 0:
        continue
    if "| --- | --- |" in text:
        continue
    if "|" in text:
        temp = text.split("|")[1:3]
        contents.append(temp)

In [ ]:
if "요인" in contents[0][0]:
    contents = contents[1:]

In [49]:
pd.DataFrame(contents, columns=['요인', '설명']).to_csv("원화_강세_요인.csv", encoding='utf-8', index=False)

In [50]:
question = """
2025년 5월 1일부터 오늘까지의의 한국어/영어/중국어 기사 중 원/달러 환율과 달러화 내용을을 조사한 후,
최근 기사에서 제시하는 원화의 통화 가치가 하락할 요인 4가지를 (요인 | 설명) [표] 형식 보여주세요.
"""

for i in range(5):
    answer = ask_le_chat(api_key, question, temperature=0.2)
    if "| --- | --- |" in answer:
        break

print(answer)

| 요인 | 설명 |
| --- | --- |
| 1. 전자금융 거래 폭발적 증가 | 전자금윤 거래의 폭발적 증가로 인해 원화 공급이 빨리 소모되고 있어 원화의 가치가 하락하고 있습니다. 이는 국내 전자금윤 거래 시장의 성장과 국제 전자금윤 거래 시장의 확대로 인해 발생하고 있습니다. |
| 2. 국내 경제 성장 느린 추세 | 국내 경제 성장이 느린 추세로 인해 원화 공급이 느려지고 있어 원화의 가치가 하락하고 있습니다. 이는 국내 생산성 증대와 소비 증가로 인해 원화 공급이 늘어나는 요인이 작아지고 있기 때문입니다. |
| 3. 외국 금융 기관의 원화 투자 감소 | 외국 금융 기관의 원화 투자가 감소하고 있어 원화의 가치가 하락하고 있습니다. 이는 외국 금융 기관의 국내 투자 수익 예측이 떨어지고 있기 때문에 발생하고 있습니다. |
| 4. 외국 경제 불안정성 증가 | 외국 경제 불안정성이 증가하고 있어 원화 요구가 늘어나고 있어 원화의 가치가 하락하고 있습니다. 이는 외국 경제 불안정성으로 인해 국내 수출 수요가 감소하고 있기 때문에 발생하고 있습니다. |


In [ ]:
if "요인" in contents[0][0]:
    contents = contents[1:]

In [52]:
contents = []
for i, text in enumerate(answer.split("\n")):
    if i == 0:
        continue
    if "| --- | --- |" in text:
        continue
    if "|" in text:
        temp = text.split("|")[1:3]
        contents.append(temp)

In [53]:
pd.DataFrame(contents, columns=['요인', '설명']).to_csv("원화_약세_요인.csv", encoding='utf-8', index=False)